<a href="https://colab.research.google.com/github/shusank8/GPT2/blob/main/GPT2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
print("GPT 2 FROM SCRATCH BY SHUSANKET BASYAL")

GPT 2 FROM SCRATCH BY SHUSANKET BASYAL


In [2]:
# starting by loading the weights just to get the understanding of it from hugging face

In [18]:
import math

In [3]:
from transformers import GPT2LMHeadModel

model_hf = GPT2LMHeadModel.from_pretrained("gpt2") # 124 M parameter
sd_hf = model_hf.state_dict()
# COOL HOW MODEL LOOKS AT THE GIVEN PROMPT AND TRIES TO CONTIUE ITS PATTERN EVEN THOUGH IT IS A PRETRAINED MODEL THAT JUST TRIES TO PREDICT THE NEXT TOKEN, IF WE HAVE EXAMPLE OF QUESTION, ANSWER IN THE PROMPT MODEL UNDERSTANDS IT AND CONTINUES TO GENERATE IN THE PATTERN OF QUESTION AND ANSWER
from transformers import pipeline
generator = pipeline("text-generation", model='gpt2')
generator("Question: What is your name? Answer: Hello, my name is shusanket basyal. Question: How old are you? Answer: I am 23 years old. Question: Where are you? Answer:I am in United States. Question: What are you doing in United States? ", max_length = 300, num_return_sequences = 5)

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/665 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/548M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/124 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/26.0 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/1.04M [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

Device set to use cuda:0
Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


[{'generated_text': 'Question: What is your name? Answer: Hello, my name is shusanket basyal. Question: How old are you? Answer: I am 23 years old. Question: Where are you? Answer:I am in United States. Question: What are you doing in United States? __________________\n\nMARK, CANDINARO, SCOI\n\nKELLEY PARK, WATERFORD, CALIFORNIA\n\n\nQ: What are you doing in United States? __________________\n\nBANK of America,\n\nNewberry Heights, WY Newberry Heights, NY, 77501\n\nDear,\n\nI\'m wondering what you do online?\n\nHave either a group job or have you a group phone? How do you see yourself? Are there any online resources to help out with your life? Any advice that I can give you or any suggestions for people to try and learn to help me out through social media? I\'m trying to find out how to use social media differently from my normal day to day lives and I would appreciate it if you could answer your questions or share this information with all you readers that read about me online. I\'m 

In [4]:
for k,v in sd_hf.items():
  print(k,v.shape)

transformer.wte.weight torch.Size([50257, 768])
transformer.wpe.weight torch.Size([1024, 768])
transformer.h.0.ln_1.weight torch.Size([768])
transformer.h.0.ln_1.bias torch.Size([768])
transformer.h.0.attn.c_attn.weight torch.Size([768, 2304])
transformer.h.0.attn.c_attn.bias torch.Size([2304])
transformer.h.0.attn.c_proj.weight torch.Size([768, 768])
transformer.h.0.attn.c_proj.bias torch.Size([768])
transformer.h.0.ln_2.weight torch.Size([768])
transformer.h.0.ln_2.bias torch.Size([768])
transformer.h.0.mlp.c_fc.weight torch.Size([768, 3072])
transformer.h.0.mlp.c_fc.bias torch.Size([3072])
transformer.h.0.mlp.c_proj.weight torch.Size([3072, 768])
transformer.h.0.mlp.c_proj.bias torch.Size([768])
transformer.h.1.ln_1.weight torch.Size([768])
transformer.h.1.ln_1.bias torch.Size([768])
transformer.h.1.attn.c_attn.weight torch.Size([768, 2304])
transformer.h.1.attn.c_attn.bias torch.Size([2304])
transformer.h.1.attn.c_proj.weight torch.Size([768, 768])
transformer.h.1.attn.c_proj.bias 

In [5]:
import torch
import torch.nn as nn
from torch.nn import functional as F

In [6]:
class GPTConfig:
  block_size = 256
  vocab_size = 65
  n_layer = 6
  n_head = 6
  n_embd = 384

In [19]:
class CausalSelfAttention(nn.Module):

  def __init__(self, config):

    super().__init__()
    assert config.n_embd % config.n_head == 0

    self.c_attn = nn.Linear(config.n_embd, 3*config.n_embd)

    self.c_proj = nn.Linear(config.n_embd, config.n_embd)

    self.n_head = config.n_head
    self.n_embd = config.n_embd

    self.register_buffer("bias", torch.tril(torch.ones(config.block_size,config.block_size)).view(1,1,config.block_size,config.block_size))

  def forward(self, x):
    B,T,C = x.size()
    qkv = self.c_attn(x)
    q,k,v = qkv.split(self.n_embd, dim=2)
    k = k.view(B,T,self.n_head, C//self.n_head).transpose(1,2)
    q = q.view(B,T,self.n_head, C//self.n_head).transpose(1,2)
    v = v.view(B,T,self.n_head, C//self.n_head).transpose(1,2)

    att = (q@k.tranpose(-2,-1))*(1.0/math.sqrt(k.size(-1)))
    att = att.masked_fill(self.bias[:,:,:T, :T]==0, float("-inf"))
    att = F.softmax(att, dim=-1)
    y = att@v
    y = y.transpose(1,2).contiguous().view(B,T,C)
    y = self.c_proj(y)
    return y

In [21]:
class MLP(nn.Module):

  def __init__(self, config):
    super().__init__()
    self.m = nn.Sequential(
    nn.Linear(config.n_embd, 4*config.n_embd),
    nn.GELU(approximate = 'tanh'),
    nn.Linear(4*config.n_embd, config.n_embd))

  def froward(self, x):
    return self.m(x)

In [22]:
class Block(nn.Module):

  def __init__(self, config):
    super().__init__()
    self.ln_1 = nn.LayerNorm(config.n_embd)
    self.attn = CausalSelfAttention(config)
    self.ln2 = nn.LayerNorm(config.n_embd)
    self.mlp = MLP(config)

  def forward(self, x):
    x = x+self.attn(self.ln1(x))
    x = x+ self.mlp(self.ln2(x))
    return x


In [ ]:
class GPT(nn.Module):

  def __init__(self, config):
    super().__init__()
    self.config = config

    self.transformer = nn.ModuleDict(dict(
        wte = nn.Embeding(config.vocab_size, config.n_embd),
        wpe = nn.Embeding(config.vocab_size, config.n_embd),
        h = nn.ModuleList([Block(config) for _ in range(config.n_layer)]),
        ln_f = nn.LayerNorm(config.n_embd)
    ))
    self.lm_head = nn.Linear(config.n_embd, config.vocab_size, bias = False)

In [23]:
print("TO BE CONTINUED.......")

TO BE CONTINUED.......
